# Image Reconstruction by Deconvolution, Denoising and Fusion

In [ ]:
from PIL import Image
from jupyter_compare_view import compare as compare_view

dtype = "float32"
import pyxu.runtime as pxrt
pxrt._setPrecision(pxrt.Width.SINGLE)

GPU = True

if GPU:
    import cupy as xp
    compare = lambda x, y, **kwargs: compare_view(x.get(), y.get(), **kwargs)
else: 
    import numpy as xp
    compare = compare_view

# Load the image
image_path = '../data/EPFL_aereal.jpeg'
ground_truth_image = Image.open(image_path)
ground_truth_image

In [ ]:
# Convert the image to a numpy array
image = xp.array(ground_truth_image)[-512:, -1024:].astype(dtype) / 255.

dim_shape = image.shape
print(dim_shape)

## Lens 


In [ ]:
import scipy.signal as ss

def get_window(name, size):
    return xp.array(ss.get_window(name, size), dtype=dtype)

In [ ]:
import pyxu.runtime as pxrt
import pyxu.operator as pxo

wsize = 20
filt = get_window('hamming', wsize)
psf = filt / filt.sum()

lens_op = pxo.Convolve(
    dim_shape=dim_shape,
    kernel=[psf, psf, xp.array(1., dtype=dtype)], 
    center=[wsize//2, wsize//2, 0], 
)

print(lens_op.lipschitz)

In [ ]:
# Convolution with PSF
data_lens = lens_op(image)

In [ ]:
compare(data_lens, image, circle_fraction=0.1)

## Pinhole photography

In [ ]:
 # Tapering
tukey_window = xp.outer(get_window('tukey', image.shape[1]), get_window('tukey', image.shape[0]))
taper_op = pxo.DiagonalOp(xp.tile(tukey_window, (3, 1, 1)).T)
taper_op.lipschitz = taper_op.estimate_lipschitz()

# Partial Masking 
mask = xp.ones(image.shape, dtype=dtype)
mask[100:250, 200:300] = 0.1
mask[250:300, 700:750] = 0.3
mask_op = pxo.DiagonalOp(vec=mask, dim_shape=dim_shape)
mask_op.lipschitz = mask_op.estimate_lipschitz()

# Compose Tapering and Masking operators
pinhole_op = mask_op * taper_op
pinhole_op.lipschitz

In [ ]:
data_pinhole = pinhole_op(image) # Low exposure 
data_pinhole = xp.clip(xp.random.normal(loc=data_pinhole, scale=xp.sqrt(data_pinhole)/4).astype(dtype), a_min=0, a_max=1) # shot noise

In [ ]:
compare(data_pinhole, image, circle_fraction=0.1)

In [ ]:
data_merge = xp.concatenate([data_pinhole[:, :500], data_lens[:, 500:]], axis=1)
compare(data_merge, image, circle_fraction=0.1)

## Pseudo-inverse Solution

In [ ]:
import pyxu.operator.blocks as pxb
sensing_op = pxb.stack([0.7*lens_op,0.3*pinhole_op])

In [ ]:
sensing_op

In [ ]:
data = xp.stack([0.7*data_lens, 0.3*data_pinhole])
data.shape

In [ ]:
adj_recon = sensing_op.adjoint(data)

In [ ]:
compare(adj_recon, data_merge, circle_fraction=0.1)

In [ ]:
import pyxu.opt.stop as pxst

stop_crit = pxst.RelError(eps=1e-3, var="x", f=None, norm=2, satisfy_all=True) | pxst.MaxIter(500)


pinv_solution = sensing_op.pinv(data, damp=0.01,
                         kwargs_init=dict(verbosity=500),
                         kwargs_fit=dict(stop_crit=stop_crit))

# use util.TO_NUMPY before saving to ZARR
# solvers --> not save by default (solver.__innit__ and solver.step)

In [ ]:
compare(pinv_solution.clip(0, 1), data_merge, circle_fraction=0.1)

## Bayesian Inversion

## Likelihood

In [ ]:
theta = 10, 0.006

loss_lens = theta[0] * pxo.SquaredL2Norm(dim_shape=dim_shape).argshift(-data_lens) * lens_op
loss_pinhole = theta[1] * pxo.L1Norm(dim_shape=dim_shape).moreau_envelope(0.01).argshift(-data_pinhole) * pinhole_op
loss = loss_lens + loss_pinhole

### Prior

In [ ]:
# Define multi-channel TV prior
grad_kwags = {"dim_shape":dim_shape, "directions":(0, 1), "accuracy":4, "mode":"constant", "gpu":GPU, "dtype":dtype}

grad = pxo.Gradient(**grad_kwags)
lambda_= .005
l21_norm = lambda_ * pxo.L21Norm(dim_shape=grad.codim_shape, l2_axis=(0, 3))
# Positivity constraint
pos_constraint = pxo.PositiveOrthant(dim_shape=dim_shape)

In [ ]:
loss.diff_lipschitz

In [ ]:
import pyxu.opt.solver as pxsol

# Stopping criterion
default_stop_crit = (pxst.RelError(eps=1e-3, var="x", f=None, norm=2, satisfy_all=True) & 
                    pxst.RelError(eps=1e-3, var="z", f=None, norm=2, satisfy_all=True) &
                    pxst.MaxIter(20)) | pxst.MaxIter(1000)

# Initialize solver (Condat-Vu primal-dual splitting algorithm in this case)
solver = pxsol.CondatVu(f=loss, g=pos_constraint, h=l21_norm, K=grad, verbosity=100)

# Fit  
solver.fit(
    x0=pinv_solution, 
    tuning_strategy=2, 
    stop_crit=default_stop_crit
)
isotv_solution = solver.solution()

In [ ]:
compare(isotv_solution.clip(0, 1), pinv_solution.clip(0, 1), circle_fraction=0.1)

### Plug and Play

In [ ]:
from pyxu.abc.operator import ProxFunc
from pyxu.info.deps import NDArrayInfo

class MedianFilter(ProxFunc):
    def __init__(self, dim_shape, filter_size=3):
        super().__init__(dim_shape=dim_shape, codim_shape=(1,))
        self._filter_size = filter_size
        
    def apply(self, arr):
        return NotImplemented
        
    def prox(self, arr, tau=None):
        if NDArrayInfo.from_obj(arr) == NDArrayInfo.NUMPY:
            import scipy.ndimage as cpu_ndimage
            median_filter = cpu_ndimage.median_filter
        if NDArrayInfo.from_obj(arr) == NDArrayInfo.CUPY:
            import cupyx.scipy.ndimage as gpu_ndimage
            median_filter = gpu_ndimage.median_filter
        return median_filter(arr, size=self._filter_size)

median_op = MedianFilter(dim_shape=dim_shape, filter_size=(3, 3, 1))

In [ ]:
solver = pxsol.PGD(f=loss, g=median_op, verbosity=100)

solver.fit(x0=pinv_solution, acceleration=True, stop_crit=pxst.MaxIter(250))
pnp_recons = solver.solution()

In [ ]:
compare(pnp_recons.clip(0, 1), isotv_solution.clip(0, 1), circle_fraction=0.1)

### Interoperability with PyTorch

In [ ]:
import torch
from torch import nn

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Dropout(0.25)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Dropout(0.25),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = Autoencoder()
model.load_state_dict(torch.load("../data/nn/ae_trained.pth"))
model.eval()
device = torch.device('cuda' if (torch.cuda.is_available() & GPU) else 'cpu')
model.to(device)

In [ ]:
from pyxu.operator.interop import from_torch

def denoiser_prox(arr, tau):
    with torch.no_grad():
        torch_input = arr.reshape(1, *dim_shape).permute(0, 3, 1, 2)
        im_denoised = model(torch_input)
        torch_output = im_denoised.squeeze().permute(1, 2, 0)
    return torch_output

nn_denoiser = from_torch(
    apply=None,
    prox=denoiser_prox,
    dim_shape=dim_shape,
    codim_shape=1,
    cls=ProxFunc,
    dtype="float32",
    enable_warnings=True,
    name='nn_denoiser',
)

In [ ]:
denoised_pinv = nn_denoiser.prox(pinv_solution, None)
compare(denoised_pinv.clip(0,1), data_merge.clip(0,1))

In [ ]:
solver = pxsol.PGD(f=loss, g=nn_denoiser, verbosity=100)

solver.fit(x0=pinv_solution, acceleration=True, stop_crit=pxst.MaxIter(100))
pytorch_recons = solver.solution()

# Change verbosity

In [ ]:
compare(pytorch_recons.clip(0, 1), data_merge)